In [1]:
TGT_CWD = '~/repos/binance-public-data/python'
import pathlib; import sys; sys.path.append(pathlib.Path(TGT_CWD).expanduser().as_posix())

In [2]:
from rsch.notebook_setup import *

setting up ipython for notebook
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


done configuring ipython

working directory : /home/tyler/repos/binance-public-data/python/rsch/notebooks

Python 3.10.13 | packaged by conda-forge | (main, Oct 26 2023, 18:07:37) [GCC 12.3.0]
Type 'copyright', 'credits' or 'license' for more information
IPython 8.15.0 -- An enhanced Interactive Python. Type '?' for help.



In [3]:
from arcticdb import Arctic
from rsch.etl import get_arcticdb_uri, get_deltalake_uri

ac = Arctic(get_arcticdb_uri())
lib_name = 'futures_um.trades.secid'

lib = ac.get_library(lib_name, create_if_missing=True)

In [4]:
btc = lib.read('BTCUSDT').data


In [8]:
import deltalake
from deltalake import write_deltalake,  DeltaTable
import duckdb


DELTA_LAKE_TABLE_PATH = get_deltalake_uri() / 'futures_um.trades.secid/BTCUSDT'
DELTA_LAKE_TABLE_PATH

# make folder if not exists
# DELTA_LAKE_TABLE_PATH.mkdir(parents=True, exist_ok=True)
# write_deltalake(DELTA_LAKE_TABLE_PATH, btc)


PosixPath('/home/tyler/repos/binance-public-data/python/data/deltalake/futures_um.trades.secid/BTCUSDT')

In [10]:
import duckdb

fpath = '/home/tyler/repos/binance-public-data/python/data/futures/um/monthly/klines/1INCHUSDT/1m/1INCHUSDT-1m-2020-12.zip'
qry = f"""
select * from read_csv('{fpath}', auto_detect=true, compression='gzip', header=true)
"""

rez = duckdb.query(qry)

IOException: IO Error: Input is not a GZIP stream

In [9]:
dt = DeltaTable(DELTA_LAKE_TABLE_PATH)
dt_pa = dt.to_pyarrow_table()
dt_pa_ds = dt.to_pyarrow_dataset()

In [10]:
len(dt_pa.column('price'))

10281248

In [12]:
btc

,trade_id,price,qty,quoteqty,time,isbuyermaker
trade_id,,,,,,
25247504,25247504,7189.4300,0.0300,215.6829,2020-01-01 00:00:01.481,True
25247505,25247505,7189.4200,1.4700,10568.4474,2020-01-01 00:00:01.481,True
25247506,25247506,7189.4200,4.2220,30353.7312,2020-01-01 00:00:01.708,True
25247507,25247507,7189.0000,0.7780,5593.0420,2020-01-01 00:00:06.290,True
25247508,25247508,7189.5000,0.6830,4910.4285,2020-01-01 00:00:06.290,True
...,...,...,...,...,...,...
35530556,35530556,9364.8600,0.4260,3989.4304,2020-01-31 23:59:56.703,False
35530557,35530557,9364.8600,0.0500,468.2430,2020-01-31 23:59:56.773,False
35530558,35530558,9364.8500,0.0100,93.6485,2020-01-31 23:59:56.780,True


In [13]:
%%timeit


qry = """
WITH PriceQty AS (
    SELECT 
        time,
        price * qty AS price_x_qty,
        qty
    FROM btc
),
Resampled AS (
    SELECT 
        date_trunc('minute', time) AS time_minute,
        SUM(price_x_qty) AS sum_price_x_qty,
        SUM(qty) AS sum_qty
    FROM PriceQty
    GROUP BY time_minute
)
SELECT 
    time_minute,
    sum_price_x_qty,
    sum_qty,
    sum_price_x_qty / NULLIF(sum_qty, 0) AS vwap
FROM Resampled
ORDER BY time_minute asc;
"""
vwap_duckdb = duckdb.sql(qry).df()
vwap_duckdb

94.2 ms ± 5.94 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [14]:
%%timeit


qry = """
WITH PriceQty AS (
    SELECT 
        time,
        price * qty AS price_x_qty,
        qty
    FROM dt_pa
),
Resampled AS (
    SELECT 
        date_trunc('minute', time) AS time_minute,
        SUM(price_x_qty) AS sum_price_x_qty,
        SUM(qty) AS sum_qty
    FROM PriceQty
    GROUP BY time_minute
)
SELECT 
    time_minute,
    sum_price_x_qty,
    sum_qty,
    sum_price_x_qty / NULLIF(sum_qty, 0) AS vwap
FROM Resampled
ORDER BY time_minute asc;
"""
vwap_duckdb = duckdb.sql(qry).df()
vwap_duckdb

93.1 ms ± 5.67 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [16]:
%%timeit


qry = f"""
WITH PriceQty AS (
    SELECT 
        time,
        price * qty AS price_x_qty,
        qty
    FROM read_parquet({dt.file_uris()})
),
Resampled AS (
    SELECT 
        date_trunc('minute', time) AS time_minute,
        SUM(price_x_qty) AS sum_price_x_qty,
        SUM(qty) AS sum_qty
    FROM PriceQty
    GROUP BY time_minute
)
SELECT 
    time_minute,
    sum_price_x_qty,
    sum_qty,
    sum_price_x_qty / NULLIF(sum_qty, 0) AS vwap
FROM Resampled
ORDER BY time_minute asc;
"""
vwap_duckdb = duckdb.sql(qry).df()
vwap_duckdb

152 ms ± 14.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [18]:
%%timeit


qry = f"""
WITH PriceQty AS (
    SELECT 
        time,
        price * qty AS price_x_qty,
        qty
    FROM dt_pa_ds
),
Resampled AS (
    SELECT 
        date_trunc('minute', time) AS time_minute,
        SUM(price_x_qty) AS sum_price_x_qty,
        SUM(qty) AS sum_qty
    FROM PriceQty
    GROUP BY time_minute
)
SELECT 
    time_minute,
    sum_price_x_qty,
    sum_qty,
    sum_price_x_qty / NULLIF(sum_qty, 0) AS vwap
FROM Resampled
ORDER BY time_minute asc;
"""
vwap_duckdb = duckdb.sql(qry).df()
vwap_duckdb

168 ms ± 9.01 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [1]:
vwap_duckdb

NameError: name 'vwap_duckdb' is not defined

In [21]:
quack = duckdb.arrow(dt_pa_ds)

In [ ]:
quack.query()

In [96]:
# rez = rez.set_index('time_bucket')
s_bar = rez.asfreq('1s').fillna({'total_volume': 0}).fillna(method='ffill')

In [99]:
s_bar.tail(20)

,open_price,high_price,low_price,close_price,total_volume
time_bucket,,,,,
2020-01-31 23:59:40,9361.0100,9361.0100,9361.0000,9361.0000,0.0000
2020-01-31 23:59:41,9361.0100,9361.0100,9361.0000,9361.0000,0.0000
2020-01-31 23:59:42,9361.0100,9361.0100,9361.0000,9361.0000,0.0000
2020-01-31 23:59:43,9361.0100,9361.0100,9361.0000,9361.0000,0.0000
2020-01-31 23:59:44,9361.0100,9361.0100,9361.0000,9361.0000,0.0000
2020-01-31 23:59:45,9361.0000,9361.0000,9361.0000,9361.0000,0.0410
2020-01-31 23:59:46,9361.0000,9361.0000,9361.0000,9361.0000,0.0000
2020-01-31 23:59:47,9361.0000,9361.0000,9361.0000,9361.0000,0.0000
2020-01-31 23:59:48,9361.0000,9361.0000,9361.0000,9361.0000,0.0000


In [101]:
s_bar.close_price.pct_change()


time_bucket
2020-01-01 00:00:01       NaN
2020-01-01 00:00:02    0.0000
2020-01-01 00:00:03    0.0000
2020-01-01 00:00:04    0.0000
2020-01-01 00:00:05    0.0000
                        ...  
2020-01-31 23:59:55    0.0001
2020-01-31 23:59:56   -0.0000
2020-01-31 23:59:57    0.0000
2020-01-31 23:59:58    0.0000
2020-01-31 23:59:59   -0.0000
Freq: S, Name: close_price, Length: 2678399, dtype: float64

In [100]:

qry = f"""
SELECT count(*) FROM dt_pa;
"""
qry = """
WITH PriceQty AS (
    SELECT 
        time,
        price * qty AS price_x_qty,
        qty
    FROM dt_pa
),
Resampled AS (
    SELECT 
        date_trunc('minute', time) AS time_minute,
        SUM(price_x_qty) AS sum_price_x_qty,
        SUM(qty) AS sum_qty
    FROM PriceQty
    GROUP BY time_minute
)
SELECT 
    time_minute,
    sum_price_x_qty,
    sum_qty,
    sum_price_x_qty / NULLIF(sum_qty, 0) AS vwap
FROM Resampled;
"""

# qry = """
# SELECT 
#     date_trunc('second', time) AS time_bucket,
#     FIRST(price) OVER w AS open_price,
#     MAX(price) OVER w AS high_price,
#     MIN(price) OVER w AS low_price,
#     LAST(price) OVER w AS close_price,
#     SUM(qty) OVER w AS total_volume
# FROM dt_pa
# WINDOW w AS (PARTITION BY date_trunc('second', time))
# --GROUP BY time_bucket
# ORDER BY time_bucket;
# """

qry = """
    SELECT
        time_bucket,
        min(price) FILTER (WHERE time = first_trade_time) as open_price,
        max(price) as high_price,
        min(price) as low_price,
        min(price) FILTER (WHERE time = last_trade_time) as close_price,
        sum(qty) as total_volume
    FROM (
        SELECT
            time,
            date_trunc('second', time) AS time_bucket,
            min(time) OVER w AS first_trade_time,
            max(time) OVER w AS last_trade_time,
            price,
            qty
        FROM dt_pa
        WINDOW w AS (PARTITION BY date_trunc('second', time))
    )
    GROUP BY time_bucket
    ORDER BY time_bucket;
"""

qry = """
WITH open_close AS (
    SELECT 
        date_trunc('second', time) AS time_bucket, 
        price AS open_price,
        time AS trade_time,
        ROW_NUMBER() OVER (PARTITION BY date_trunc('second', time) ORDER BY time) AS rn,
        ROW_NUMBER() OVER (PARTITION BY date_trunc('second', time) ORDER BY time DESC) AS rn_desc
    FROM dt_pa
),
aggregations AS (
    SELECT 
        date_trunc('second', time) AS time_bucket,
        MAX(price) AS high_price,
        MIN(price) AS low_price,
        SUM(qty) AS volume
    FROM dt_pa 
    GROUP BY date_trunc('second', time)
)
SELECT 
    a.time_bucket,
    MIN(oc.open_price) FILTER (WHERE oc.rn = 1) as open_price,
    a.high_price,
    a.low_price,
    MIN(oc.open_price) FILTER (WHERE oc.rn_desc = 1) as close_price,
    a.volume AS total_volume
FROM aggregations a
JOIN open_close oc ON a.time_bucket = oc.time_bucket
GROUP BY a.time_bucket, a.high_price, a.low_price, a.volume
ORDER BY a.time_bucket;

"""

qry = """
WITH open_close AS (
    SELECT 
        date_trunc('second', time) AS time_bucket, 
        price AS open_price,
        time AS trade_time,
        ROW_NUMBER() OVER (PARTITION BY date_trunc('second', time) ORDER BY time) AS rn,
        ROW_NUMBER() OVER (PARTITION BY date_trunc('second', time) ORDER BY time DESC) AS rn_desc
    FROM dt_pa
),
aggregations AS (
    SELECT 
        date_trunc('second', time) AS time_bucket,
        MAX(price) AS high_price,
        MIN(price) AS low_price,
        SUM(qty) AS volume
    FROM dt_pa 
    GROUP BY date_trunc('second', time)
),
filled_aggregations AS (
    SELECT 
        s.time AS time_bucket,
        COALESCE(ag.high_price, LAG(ag.high_price) OVER (ORDER BY s.time)) AS high_price,
        COALESCE(ag.low_price, LAG(ag.low_price) OVER (ORDER BY s.time)) AS low_price,
        COALESCE(ag.volume, 0) AS volume
    FROM all_seconds s 
    LEFT JOIN aggregations ag ON s.time = ag.time_bucket
)
SELECT 
    f.time_bucket,
    COALESCE(MIN(oc.open_price) FILTER (WHERE oc.rn = 1), LAG(MIN(oc.open_price) FILTER (WHERE oc.rn_desc = 1)) OVER (ORDER BY f.time_bucket)) as open_price,
    f.high_price,
    f.low_price,
    COALESCE(MIN(oc.open_price) FILTER (WHERE oc.rn_desc = 1), LAG(MIN(oc.open_price) FILTER (WHERE oc.rn_desc = 1)) OVER (ORDER BY f.time_bucket)) as close_price,
    f.volume AS total_volume
FROM filled_aggregations f
LEFT JOIN open_close oc ON f.time_bucket = oc.time_bucket
GROUP BY f.time_bucket, f.high_price, f.low_price, f.volume
ORDER BY f.time_bucket;

"""

# qry = """
# SELECT 
#     date_trunc('second', time) AS time_bucket,
#     FIRST_VALUE(price) OVER w AS open_price,
#     MAX(price) OVER w AS high_price,
#     MIN(price) OVER w AS low_price,
#     LAST_VALUE(price) OVER w AS close_price,
#     SUM(qty) OVER w AS total_volume
# FROM dt_pa
# WINDOW w AS (PARTITION BY date_trunc('second', time) 
#               ORDER BY time 
#               ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING)
# GROUP BY time_bucket
# ORDER BY time_bucket;

# """

# qry = """
# SELECT 
#     date_trunc('second', time) AS time_bucket,
#     FIRST_VALUE(price) OVER (PARTITION BY date_trunc('second', time) ORDER BY time ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS open_price,
#     MAX(price) OVER (PARTITION BY date_trunc('second', time) ORDER BY time ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS high_price,
#     MIN(price) OVER (PARTITION BY date_trunc('second', time) ORDER BY time ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS low_price,
#     LAST_VALUE(price) OVER (PARTITION BY date_trunc('second', time) ORDER BY time ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS close_price
# FROM dt_pa
# ORDER BY time_bucket;

# """
rez = duckdb.sql(qry).df()


time_bucket
2020-01-01 00:00:01       NaN
2020-01-01 00:00:02    0.0000
2020-01-01 00:00:03    0.0000
2020-01-01 00:00:04    0.0000
2020-01-01 00:00:05    0.0000
                        ...  
2020-01-31 23:59:55    0.0001
2020-01-31 23:59:56   -0.0000
2020-01-31 23:59:57    0.0000
2020-01-31 23:59:58    0.0000
2020-01-31 23:59:59   -0.0000
Freq: S, Name: close_price, Length: 2678399, dtype: float64

CatalogException: Catalog Error: Table with name all_seconds does not exist!
Did you mean "temp.information_schema.tables"?

In [82]:
# rez = rez.set_index('time_bucket')
rez.asfreq('s')

,open_price,high_price,low_price,close_price,total_volume
time_bucket,,,,,
2020-01-01 00:00:01,7189.4300,7189.4300,7189.4200,7189.4200,5.7220
2020-01-01 00:00:02,NaN,NaN,NaN,NaN,NaN
2020-01-01 00:00:03,NaN,NaN,NaN,NaN,NaN
2020-01-01 00:00:04,NaN,NaN,NaN,NaN,NaN
2020-01-01 00:00:05,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
2020-01-31 23:59:55,9365.6600,9366.7400,9364.9400,9364.9400,23.6400
2020-01-31 23:59:56,9364.9600,9364.9600,9364.8500,9364.8500,5.7860
2020-01-31 23:59:57,NaN,NaN,NaN,NaN,NaN


In [1]:
rez

NameError: name 'rez' is not defined

In [58]:
dt_pa

pyarrow.Table
trade_id: int64
price: double
qty: double
quoteqty: double
time: timestamp[us]
isbuyermaker: bool
__index_level_0__: int64
----
trade_id: [[25247504,25247505,25247506,25247507,25247508,...,25313035,25313036,25313037,25313038,25313039],[25313040,25313041,25313042,25313043,25313044,...,25378571,25378572,25378573,25378574,25378575],...,[35407393,35407394,35407395,35407396,35407397,...,35472924,35472925,35472926,35472927,35472928],[35472929,35472930,35472931,35472932,35472933,...,35530556,35530557,35530558,35530559,35530560]]
price: [[7189.43,7189.42,7189.42,7189,7189.5,...,7220.88,7220.88,7220.87,7220.39,7220.4],[7220.39,7220.37,7220.4,7220.42,7220.42,...,7140.8,7140.13,7140.97,7140.07,7140.05],...,[9270.96,9270.96,9270.96,9270.96,9270.96,...,9287.14,9287.14,9287.14,9287.13,9287.13],[9287.14,9287.14,9287.13,9287.13,9287.07,...,9364.86,9364.86,9364.85,9364.85,9364.51]]
qty: [[0.03,1.47,4.222,0.778,0.683,...,0.017,0.417,0.417,0.74,0.5],[0.06,0.44,0.02,0.05,0.007,...,0.1,0.005,

In [45]:
rez.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44640 entries, 36453 to 14650
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   time_minute      44640 non-null  datetime64[us]
 1   sum_price_x_qty  44640 non-null  float64       
 2   sum_qty          44640 non-null  float64       
 3   vwap             44640 non-null  float64       
dtypes: datetime64[us](1), float64(3)
memory usage: 1.7 MB


In [50]:
# %%timeit

rez = duckdb.sql(qry).arrow()


In [49]:
%%timeit

rez = duckdb.sql(qry).df()


73.6 ms ± 4.43 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [52]:
df = rez.to_pandas()

In [56]:
type(df)
df.info()
df.dtypes

pandas.core.frame.DataFrame

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44640 entries, 0 to 44639
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   time_minute      44640 non-null  datetime64[ns]
 1   sum_price_x_qty  44640 non-null  float64       
 2   sum_qty          44640 non-null  float64       
 3   vwap             44640 non-null  float64       
dtypes: datetime64[ns](1), float64(3)
memory usage: 1.4 MB


time_minute        datetime64[ns]
sum_price_x_qty           float64
sum_qty                   float64
vwap                      float64
dtype: object